# Encuesta de Hospitales Exploration

## Import Libraries

In [7]:
import pandas as pd
import csv

In [8]:

_enHospObj = pd.ExcelFile('dataset/Encuesta Nacional de Hospitales versión Diálisis(Responses).xlsx')
enHospDf = pd.read_excel('dataset/Encuesta Nacional de Hospitales versión Diálisis(Responses).xlsx',sheet_name=_enHospObj.sheet_names[0])


## First Exploration iteration

In [9]:
# 118 Columns

# enHospDf.info() # float(10 columns), int(64 columns), object(93 columns)

enHospDf.isna().sum().sum() #17,539 null datapoints

nullValues = enHospDf.isnull().mean().sort_values(ascending = False) # Percentage of Null Values per column

nullValues[nullValues > 0.1].count() # 46 columns have more than 10% null Values

nullValues.head() # Top 5 Variables have more than 87% of their values missing


61.- En caso que haya ocurrido algún tipo de protesta, señale las opciones de protesta mas ajustadas  [Otros]       0.936675
15.- ¿Cuál es la/las razón(es) por la que no está operativo el servicio de diálisis? (seleccione las correctas)     0.908971
14.- ¿Desde cuándo no está operativo el servicio de diálisis?                                                       0.895778
30.- ¿Cuál es la/las razón(es) por la que no está operativo el servicio de nutrición? (seleccione las correctas)    0.873351
29.- ¿Desde cuándo no está operativo el servicio de nutrición?                                                      0.873351
dtype: float64

### Group Variables by heuristics
 The dataset is divided in 11 different sections:
 

1. ID Data
2. General operability Var 5 to var 8
3. Specific operability <br> - In this section the 3 questions are likert scales. Thus, the high number of columns 
4. Dialysis Service
5. Nutrition Service
6. Attention Quality
7. Personnel Availability
8. Public Service
9. Mortality Impact
10. Violence and Protests
11. Epidemiological Emergency

In [108]:
# Section 0
idData = enHospDf.iloc[:,0:6]

# Section 1
genOp = enHospDf.iloc[:,6:10]

# Section 2
espOp = enHospDf.iloc[:,10:47]

# Section 3
dialServ = enHospDf.iloc[:,47:72]

# Section 4
nutriServ = enHospDf.iloc[:,72:80]

# Section 5
attQual = enHospDf.iloc[:,80:82]

# Section 6
personelDisp = enHospDf.iloc[:,82:91]

# Section 7 
publicServ = enHospDf.iloc[:,91:104]

# Section 8
mortImpact = enHospDf.iloc[:,104:108]

# Section 9 
violence = enHospDf.iloc[:,108:114]

# Section 10 
epiEmerg = enHospDf.iloc[:,115]

#### General Operability 

##### This section contains: 
1. <b>Number of architectural beds</b>: Maximum number of beds that the hospital was designed for. 
<br>
2. <b>Number of operative beds</b>: Number of beds that are currently functional.
<br>
3. <b>Number of operative emergency beds</b>: Number of functional beds in ER.
<br>
4. <b>Number of operative hospital pavilions</b>:
 - Pavilion Definition: One of a group of related buildings (cambridge dictionary)
 <br><br>

##### Possible Business Questions: <br>
This section portrays the most elemental hospital operativity.

- What are the current conditions of the hospital in terms of its operative beds and pavilions?
- How the original number of beds differ from the actual number of beds that are operative?
- Which hospitals have the highest amount of operative beds and Pavilions? 
- Which states have the highest/lowest amount of operative beds and Pavilions? 
<br><br> 

##### Questions to Client

- The the sum of operative beds and operative emergency beds is higher the number of architectural beds. If the number of architectural beds is the maximum amount of beds that the hospital was designed for why the sum of all the beds is higher? 


<br><br> 

Source: http://maiquiflores.over-blog.es/article-definiciones-basicas-de-los-indicadores-de-la-gestion-hospitalaria-en-venezuela-2014-123693467.html

Better source pending

In [12]:
# genOp.head()

# Row 615 has corrupt datapoint. -> 8|8
# Check with client how the quality of data that is submitted.
_corruptGenOp = genOp[genOp['5.- Número de camas arquitectónicas'].apply(lambda x: type(x) == str)] 

genOp = genOp.drop(_corruptGenOp.index).astype(int)


In [13]:
# 131 rows where the sum of operative beds and operative emergency beds is higher to number of arquitectural beds

genOp[(genOp.iloc[:,1] + genOp.iloc[:,2]) > genOp.iloc[:,0]]

,5.- Número de camas arquitectónicas,6.- Número de camas operativas,7.- Número de camas operativas en emergencia,8.- Número de pabellones operativos
0,15,6,18,3
1,265,240,30,3
5,160,180,20,4
14,180,160,50,4
17,160,180,20,4
...,...,...,...,...
741,196,150,89,2
743,200,179,27,3
755,104,91,91,3
756,104,92,92,3


#### Specific Operability 
No null values
<br> 

##### Possible Questions
- What is the equipment that lacks the most?
- Which hospitals have the most scarce equipment?
- Which hospitals have the most complete equipment?
- What are the medicines that lack the most?
- What are the medicines that hospitals have the most availability?
- Which hospitals have most scarcity of medicines?
- Which hospitals have the most amount of medicines? 
- What is the availability of medicines per hospital? (interactive viz)
- What is the availability of medicines per state? 
    - <i>In order to track demand of medicines as well</i>
<br>


##### Variables

###### 9.- Equipment <br>
- UTI: The UTI is the Intermediate Treatment Unit that has specialized personnel and equipment that allows patients who, after the critical stage, require constant surveillance, attention and observation, prior to transfer to a conventional room.
- UTI(P): Definition Pending
- Emergency: How much time has the ER been operative.
- Pavilion: How much time have the Pavilions been operative.
- Laboratory: How much time has the Laboratory been operative. 
- Ecography: How much time has the ecography been operative. 
- TAC/TMN: Axial Computarized Tomography [TAC], TMN To be defined. 
- X Rays: How much time has the X-Ray equipment has been operative. 


###### 10.- Medicines <br> 
- Adrenalin:
- Atropine:
- Dopamine:
- Cephalosporins / beta-lactams:
- Aminoglycosides / quinolones:
- Vancomycin / Clindamycin:
- Local anesthesia (lidocaine:
- Minor analgesic (NSAIDs, dipyrone, etc.:
- Major analgesic (morphine, demerol:
- Fluidotherapics (0.9%, Gluc 5%,) / infusion equipment:
- Diazepam / DPH:
- Heparin:
- Steroids:
- Insulin:
- Med inhaled for asthma:
- Antihypertensive:
- Defibrillator:
- TOT / intubation:
- Yelco / central roads:
- Oxygen / suction:
- Minor analgesics:
- Major analgesics:
- Anesthetic gases:
- EV anesthetics:
- Relaxing:
- TOT / intubation equipment:
- Patient lingerie kit:
- Disposable health personnel (cap, gloves, gown:
- Oxygen / suction:


In [37]:
# from PyDictionary import PyDictionary

# dictionary = PyDictionary() # In case we need to look up for the medicine definitions. 

# dictionary.meaning('Cephalosporins')
espOp.head()

,9.- Operatividad de unidades específicas [UTI],9.- Operatividad de unidades específicas [UTI(P)],9.- Operatividad de unidades específicas [Emergencia],9.- Operatividad de unidades específicas [Pabellón],9.- Operatividad de unidades específicas [Laboratorio],9.- Operatividad de unidades específicas [Ecografía],9.- Operatividad de unidades específicas [TAC/RMN],9.- Operatividad de unidades específicas [Rayos x],10.- Señale insumos disponibles en emergencia. Lista de insumos [Adrenalina],10.- Señale insumos disponibles en emergencia. Lista de insumos [Atropina],...,10.- Señale insumos disponibles en emergencia. Lista de insumos [Oxígeno / succión],11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos [Analgésicos menores],11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos [Analgésicos mayores],11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos [Gases anestésicos],11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos [Anestésicos EV],11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos [Relajantes],11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos [TOT / equipo intubación ],11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos [Kit lencería paciente],"11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos [Desechables personal de salud( tapaboca, guantes , bata )]",11.- Señale insumos quirúrgicos disponibles (apendicitis). Lista de insumos [Oxígeno / succión]
0,Menos de 3 de días,Menos de 3 de días,Entre 3 y 5 días,Entre 3 y 5 días,No operativa,Entre 3 y 5 días,No operativa,No operativa,No hubo,No hubo,...,Todos los días,No hubo,No hubo,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días
1,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Todos los días,Todos los días,...,Todos los días,Entre 3 y 5 días,Entre 3 y 5 días,Todos los días,Todos los días,Entre 3 y 5 días,Todos los días,Entre 3 y 5 días,Entre 3 y 5 días,Todos los días
2,Entre 3 y 5 días,Nunca ha existido,Entre 3 y 5 días,Entre 3 y 5 días,No operativa,Entre 3 y 5 días,Nunca ha existido,No operativa,Todos los días,Todos los días,...,Entre 1 y 2 días,Entre 3 y 5 días,Entre 1 y 2 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 1 y 2 días,Entre 1 y 2 días,Entre 1 y 2 días
3,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,No operativa,No operativa,No operativa,No operativa,Todos los días,Todos los días,...,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días
4,No operativa,Nunca ha existido,No existe,No existe,No operativa,No operativa,Nunca ha existido,No existe,Todos los días,Todos los días,...,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días


#### Dialysis Service 
<br> 
<i> Note: All hospitals that don't have a dialysis service will not have datapoints in the rest of the section's questions.178 rows don't have dialysis service<i>  
<br>

##### Overall Dialysis Service
- How many hospitals count with a dialysis service?
- How many hospitals don't count with a dialysis service? 

<br>

##### Non-Operative Dialysis Service
- How many hospitals have a dialysis service that is not operative?
- Why these hospitals dialysis service is not operative? 
- What is the name/location of these hospitals that don't have operative dialysis.  

<br>

##### Operative Dialysis Service
<i> Note: Peritoneal and Hemodialysis are focused on the hospitals that provide them rather than the number of patients that each hospital attend. The reasoning behind this is to clearly communicate the overall state of dialysis services in Venezuela.   
<br> 
- Overall Dialysis Service:
    - How many and which hospitals have an operative dialysis service
    - What is the number of weekly dialysis patients that each hospital has attended across time? 
        - <i> This question is designed to inspect the patient service trend across time <i>
    - What is the total average number of hemodialysis and peritoneal dialysis patients by hospital?
- Peritoneal Dialysis
    - What is the number of hospitals that have peritoneal dialysis out of the total?
    - What are the reason why hospitals don't provide peritoneal dialysis?
    - Which hospitals don't provide peritoneal dialysis?
- Hemodialysis
    - What is the number of hospitals that have peritoneal dialysis out of the total? 
    - What is the overall percentage of acute and chronic hemodialysis patients? 

<br> 
##### Dialysis Materials Availability

- Questions: 
    - What are the materials that lack the most?
    - What are the materials that hospitals have the most availability?
    - Which hospitals have most scarcity of materials?
    - Which hospitals have the most amount of materials? 
    - What is the availability of materials per hospital? (interactive viz)
    - What is the availability of materials per state? 
        - <i>In order to track demand of medicines as well</i>
    - How many and which hospitals have an operative inverse osmosis plant?
<br>


###### Type of Materials
- Filters
- Connection lines
- Hemodialysis kit
- Iron
- Complex B
- Calcium
- Zemblar®

<br>

##### Dialysis Personnel Availability
- Questions: 
    - What personnel lacks the most? 
    - What personnel has the most availability?
    - Which hospitals have the most lack & availability of dialysis personal?
    - Which states have the most lack & availability of dialysis personal?
    - What is the average number of days that each type of personnel is missing by hospital and by state?



###### Type of DIalysis Personnel
- Nephrologist Specialist
- General practitioner
- Resident
- Graduate nurse
- Nurse specialist in nephrology


In [103]:
dialServ

# Hospitals that never had dialysis service
# dialServ[dialServ['12.- ¿Existe servicio de diálisis en su hospital?'] == 'No'].isnull().sum()

# Hospitals that have a dialysis service
# dialServ[dialServ['12.- ¿Existe servicio de diálisis en su hospital?'] == 'Sí']

# Hospitals that have dialysis service that is not operative
# dialServ[(dialServ['12.- ¿Existe servicio de diálisis en su hospital?']=='Sí') & (dialServ['13.-¿Está operativo el servicio de diálisis?'] == 'No')]


,12.- ¿Existe servicio de diálisis en su hospital?,13.-¿Está operativo el servicio de diálisis?,14.- ¿Desde cuándo no está operativo el servicio de diálisis?,15.- ¿Cuál es la/las razón(es) por la que no está operativo el servicio de diálisis? (seleccione las correctas),16.- ¿Cuál es la cantidad de pacientes promedio diario en diálisis en el servicio en la última semana?,17.- ¿Cuántos de estos están en diálisis peritoneal?,"18.- Si la respuesta a la pregunta anterior es ""0"" ¿Cuál es la/las razón(es) por la que no se realiza diálisis peritoneal? (seleccione las correctas)",19.- ¿Cuántos de estos están en hemodiálisis?,20.-¿Cuántos de estos pacientes son agudos?,21.-¿Cuántos de estos pacientes son crónicos?,...,24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Hierro],24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Complejo B],24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Calcio],24.- De los materiales médico-quirúrgicos y de laboratorio necesarios para la hemodiálisis describa la disponibilidad en la última semana [Zemblar®],"25.-Con respecto a la planta de ósmosis inversa, ¿Cuántos días estuvo operativa la última semana?",26.- Con respecto al personal que trabajó en el servicio de diálisis la última semana. ¿Cuántos días trabajo cada profesional? [Especialista Nefrólogo],26.- Con respecto al personal que trabajó en el servicio de diálisis la última semana. ¿Cuántos días trabajo cada profesional? [Médico general],26.- Con respecto al personal que trabajó en el servicio de diálisis la última semana. ¿Cuántos días trabajo cada profesional? [Residente],26.- Con respecto al personal que trabajó en el servicio de diálisis la última semana. ¿Cuántos días trabajo cada profesional? [Enfermera graduada],26.- Con respecto al personal que trabajó en el servicio de diálisis la última semana. ¿Cuántos días trabajo cada profesional? [Enfermera especialista en nefrología]
0,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sí,Sí,NaN,NaN,10.0,1.0,NaN,NaN,NaN,NaN,...,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días
2,Sí,Sí,NaN,NaN,6.0,0.0,"Falta de material quirúrgico, Falta de insumos...",6.0,1.0,5.0,...,Entre 3 y 5 días,Entre 1 y 2 días,Entre 1 y 2 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Todos los días,Entre 3 y 5 días,Entre 3 y 5 días,No hubo
3,Sí,Sí,NaN,NaN,41.0,4.0,NaN,25.0,5.0,6.0,...,Todos los días,Todos los días,Todos los días,Todos los días,Ningún día,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días,Entre 3 y 5 días
4,Sí,Sí,NaN,NaN,24.0,0.0,Falta de insumos médicos,0.0,0.0,24.0,...,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Nunca ha existido,Nunca ha existido,Todos los días,Todos los días
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,Sí,Sí,NaN,NaN,14.0,0.0,"Falta de material quirúrgico, Falta de insumos...",NaN,NaN,NaN,...,Todos los días,Todos los días,Todos los días,Todos los días,Entre 3 y 5 días,Todos los días,Nunca ha existido,Todos los días,Entre 1 y 2 días,No hubo
754,Sí,Sí,NaN,NaN,15.0,0.0,Falta de material quirúrgico,15.0,0.0,15.0,...,Entre 1 y 2 días,Entre 1 y 2 días,Entre 1 y 2 días,No hubo,Todos los días,Entre 1 y 2 días,Nunca ha existido,Entre 1 y 2 días,Entre 1 y 2 días,Entre 1 y 2 días
755,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Nutrition Service
- Questions:
    - Overall Nutrition Service
        - How many hospitals out of the total count with nutrition service?
        - Which hospitals don't count with nutrition service in the last week?
        - From the hospitals that count with nutrion service how many don't have an operative nutrion service?
        - From these hospitals, what is the most frequent reason that the nutrition service is not operative? 
        - Which hospitals have a nutrition service that is not operative in the last week?  
        - What is the average operability of the nutrition service in terms of time?
    - Meals Frequency: 
        - How frequent are the daily meals overall in terms of time?
        - Which hospitals don't count with a nutrition service in the last week? 
        - Which hospitals have a low frequency of daily meals?
    - Meals quality: 
        - What is the quality of nutrition service across time? 
        - What is the quality of nutrition service of all hospitals in the last week? 
        - Which have the lowest quality of nutrition service in the last week? 
    - Baby Milk Formulas: 
        - What is the overall frequency of the supply of milk formulas for babies? 
        - Which hospitals have the lowest/highest frequency of baby formulas in the past week? 

In [119]:
# nutriServ['27.- ¿Existe servicio de nutrición en su hospital?'].value_counts()
# nutriServ[nutriServ['27.- ¿Existe servicio de nutrición en su hospital?'] == 'No'].isnull().sum()

# nutriServ[(nutriServ['27.- ¿Existe servicio de nutrición en su hospital?'] == "Sí") & (nutriServ['28.-¿Está operativo el servicio de nutrición?'] == 'No')]
nutriServ

,27.- ¿Existe servicio de nutrición en su hospital?,28.-¿Está operativo el servicio de nutrición?,29.- ¿Desde cuándo no está operativo el servicio de nutrición?,30.- ¿Cuál es la/las razón(es) por la que no está operativo el servicio de nutrición? (seleccione las correctas),31.- ¿Cuántos días funcionó el servicio de nutrición la última semana?,32.- Frecuencia diaria de comidas del servicio de nutrición,33.- Calidad del servicio de nutrición,34. - Frecuencia en el suministro de fórmulas lácteas (solo si existe pediatría)
0,Sí,Sí,NaN,NaN,Entre 3 y 5 días,Se sirven menos de 3 comidas al día,No se siguen recomendaciones del especialista ...,NaN
1,Sí,Sí,NaN,NaN,Todos los días,"Se sirve 3 comidas al día (desayuno, almuerzo...",No se siguen recomendaciones del especialista ...,Nunca ha habido fórmulas lácteas
2,Sí,Sí,Menos de una semana,"Falta de insumos, Falta de equipos, Falta de p...",Entre 3 y 5 días,Se sirven menos de 3 comidas al día,No se siguen recomendaciones del especialista ...,Hubo fórmulas lácteas entre 3 y 5 días
3,Sí,Sí,NaN,NaN,Todos los días,Se sirven menos de 3 comidas al día,No se siguen recomendaciones del especialista ...,No hubo fórmulas lácteas ningún día
4,Sí,Sí,NaN,NaN,Todos los días,"Se sirve 3 comidas al día (desayuno, almuerzo...",Se siguen recomendaciones del especialista en ...,Hubo fórmulas lácteas todos los días
...,...,...,...,...,...,...,...,...
753,Sí,Sí,NaN,NaN,Todos los días,"Se sirve 3 comidas al día (desayuno, almuerzo...",Se siguen recomendaciones del especialista en ...,Hubo fórmulas lácteas todos los días
754,Sí,Sí,NaN,NaN,Todos los días,Se sirven menos de 3 comidas al día,No se siguen recomendaciones del especialista ...,Hubo fórmulas lácteas entre 3 y 5 días
755,Sí,Sí,NaN,NaN,Todos los días,"Se sirve 3 comidas al día (desayuno, almuerzo...",No se siguen recomendaciones del especialista ...,No hubo fórmulas lácteas ningún día
756,Sí,Sí,NaN,NaN,Todos los días,"Se sirve 3 comidas al día (desayuno, almuerzo...",No se siguen recomendaciones del especialista ...,No hubo fórmulas lácteas ningún día


### First Translation
This is a quick translation of the columns using google translate's API

In [224]:
from googletrans import Translator
translator = Translator()
translation = translator.translate(enHospDf.columns.to_list(), src= 'spanish', dest='english')


In [227]:
# Append translated columns to list
transCols = []

for i in range(enHospDf.shape[1]): 
    transCols.append(translation[i].text)
    
# create csv with translated columns
# with open ('dataset/translatedCols_1.csv','w') as myfile:
#     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#     wr.writerow(transCols)

In [250]:
# Create new DF with translated columns
enHospDfTrans = enHospDf.copy() # Copy original DF
enHospDfTrans.columns = transCols # Overwrite Column Names

# Write Csv
# enHospDfTrans.to_csv('dataset/NationalSurveyHospitals_Translated.csv',index = False)

enHospDfTrans.head()

,timestamp,reporting week,1. Code of the Hospital,2. Federal Entity,3. Type Hospital,4. Administrative Entity (select all that apply),5. Number of architectural beds,6. Number of operating beds,7. Number of operating beds in emergency,8. Number of operating pavilions,...,"61. Should some sort of protest, indicate the options tighter protest occurred [Medical staff]","61. In case any kind of protest occurred, indicate the options protest tighter [Nurses]","61. Should some sort of protest, indicate the options tighter protest occurred [Other personnel of the Hospital]","61. In case any kind of protest occurred, indicate the options protest tighter [Family and / or patients]","61. In case any kind of protest occurred, indicate the options protest tighter [Other]","62.- Should have occurred situations of violence against hospital staff, indicate the corresponding options (select all that apply)",63. Point out any situation (deceased or hospitalized) with clinical suspicion of any of these conditions (select all that apply),24.- From the medical-surgical and laboratory supplies necessary for hemodialysis describe availability in the last week [Catheters high flow],"24. Of the materials medical surgical and laboratory needed for hemodialysis describe availability in the last week [serology tests (HIV, HVB, HVC and VDR) L)]",24.- From the medical-surgical and laboratory supplies necessary for hemodialysis describe availability in the last week [immediate access% reduction in urea or BUN]
0,2019-05-31 02:24:50.176,22 del 2019,DCA001,Dtto Capital,Tipo III,MPPS,15,6,18,3,...,NaN,NaN,NaN,NaN,NaN,"Robos , hurtos o disparos dentro del centro as...",TBC en personal de salud,NaN,NaN,NaN
1,2019-06-01 14:10:32.817,22 del 2019,DCA005,Dtto Capital,Tipo IV,IVSS,265,240,30,3,...,NaN,NaN,NaN,NaN,NaN,Violencia contra personal de hospital por fami...,"Difteria, Sarampión, Síndrome Gillan Barré",NaN,NaN,NaN
2,2019-06-03 18:18:40.727,22 del 2019,POR000,Portuguesa,Tipo III,MPPS,300,240,45,33,...,NaN,NaN,NaN,NaN,NaN,"Robos , hurtos o disparos dentro del centro as...",TBC en personal de salud,NaN,NaN,NaN
3,2019-06-03 18:44:38.682,22 del 2019,FAL000,Falcón,Tipo IV,Gob. Estatal,300,211,30,1,...,Protestas sin suspensión de actividades,Protestas sin suspensión de actividades,NaN,Protestas sin suspensión de actividades,NaN,Violencia contra personal de hospital por fami...,NaN,NaN,NaN,NaN
4,2019-06-03 18:52:46.912,22 del 2019,VAR000,Vargas,Tipo III,IVSS,180,120,14,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
